In [33]:
import requests
import re
from bs4 import BeautifulSoup
import json

def get_property(list_of_url):
    full_house_dict = {}
    for url in list_of_url:
        response = requests.get(url)
        if response.status_code == 200:
            html_content = response.text
            soup = BeautifulSoup(html_content, "lxml")
            script_tag = soup.find("script", string=lambda text: text and "window.classified" in text)
            if script_tag:
                script_content = script_tag.string.strip()
                pattern = r"window\.classified = ({.*?});"
                match = re.search(pattern, script_content)
                if match:
                    json_data = match.group(1)
                    house_dict = json.loads(json_data)
                    keys_to_delete = ["cluster", "customers", "features", "premiumProjectPage", "media", "alternativeDescriptions", "hasSectionsArray", "displayFlags"]
                    house_dict = {key: value for key, value in house_dict.items() if key not in keys_to_delete}
                    full_house_dict[house_dict['id']] = house_dict
        else:
            print(f"Errorcode: {response.status_code}")

    return full_house_dict

def extract_nested_values(data, keys_to_extract, prefix=""):
    results = []
    if isinstance(data, dict):
        for key, value in data.items():
            new_prefix = f"{prefix}.{key}" if prefix else key
            if key in keys_to_extract:
                results.append((new_prefix, value))
            results.extend(extract_nested_values(value, keys_to_extract, prefix=new_prefix))
    return results

def get_urls(num):
    list_all_url = []
    count = 0
    for i in range(1, num + 1):
        root_url = f"https://www.immoweb.be/en/search/house/for-sale?countries=BE&page={i}&orderBy=relevance"
        req = requests.get(root_url)
        content = req.content
        soup = BeautifulSoup(content, "html")
        
        if req.status_code == 200:
            for tag in soup.find_all("a", attrs={"class": "card__title-link"}):
                list_all_url.append(tag.get("href"))
                count += 1
        else:
            print("page not found")
            break
    print(f"Number of houses: {count}")

    return list_all_url

def filter_house_dict(house_dict):
    filtered_data = {}
    for key in house_dict:
        kitchen_type = house_dict[key]['property']['kitchen']['type']
    list_key_values = [
        "locality", "type", "subtype", "mainValue", "type_of_sale", "bedroomCount",
        "netHabitableSurface", kitchen_type, "isFurnished", "fireplaceExists", "hasTerrace",
        "hasgarden", "surface", "facadeCount", "hasSwimmingPool", "condition"
    ]
    filtered_data[house_dict["id"]] = extract_nested_values(house_dict, list_key_values)
    return filtered_data

list_of_urls = get_urls(1)
house_dict = get_property(list_of_urls)
print(house_dict)
#filtered_data = filter_house_dict(house_dict)
#print(filtered_data)


Number of houses: 60
Errorcode: 404
{10666853: {'id': 10666853, 'flags': {'isPublicSale': False, 'isNewClassified': True, 'isNewPrice': False, 'isInvestmentProject': False, 'isNewlyBuilt': False, 'isNotarySale': None, 'isLifeAnnuitySale': False, 'adQuality': None, 'date': None, 'priceSqm': None, 'price': None, 'default': None, 'isSoldOrRented': False, 'isLowEnergy': None, 'percentSold': None, 'isPassiveHouse': None, 'isNewRealEstateProject': False, 'isAnInteractiveSale': None, 'isUnderOption': False}, 'property': {'type': 'HOUSE', 'subtype': 'HOUSE', 'title': 'Construction 2020, 3CH + terasses, garage, PEB : B.', 'description': "Nigel IMMO a le plaisir de vous présenter cette maison Bel-étage neuve 2 façades d'une superficie totale de 223 m² située dans un quartier résidentiel proche de toutes commodités. Cette maison moderne se compose d'un rez-de-chaussée comprenant un garage pouvant accueillir deux voitures, deux pièces de rangements, un hall d'entrée. Le premier étage dispose d'un 